In [15]:
import os
import pandas as pd
import json

In [16]:
# 폴더에 있는 모든 파일을 부른다
path_dir = "./"
file_list = [path_dir + file for file in os.listdir(path_dir) if file.endswith(".csv")]
file_list

['./1.csv',
 './2.csv',
 './output (3).csv',
 './output (4).csv',
 './output (5).csv']

In [17]:
# json list 형태 저장
json_list = []
for file in file_list:
    with open(file, 'r') as f:
        json_list.append(json.load(f))
print(len(json_list))

5


In [18]:
# 정답 key list 저장
key_list = list(json_list[0].keys())
len(key_list)

600

In [19]:
# data를 합친다
# best가 data0에 들어가있음(현재)
df_list = []
column_name_list = []
for i, json in enumerate(json_list):
    df = pd.DataFrame([json])
    df = df.transpose()
    column_name = 'data'+str(i)
    df.rename(columns={0:column_name},inplace=True)
    df_list.append(df)
    column_name_list.append(column_name)
    
merge_df = pd.concat(df_list,axis=1)

In [20]:
merge_df

,data0,data1,data2,data3,data4
mrc-1-000653,40억년전 지구,지구,40억년전 지구,사락사라,별하늘계의 변경에 위치한 외계행성 사만(サマーン)
mrc-1-001113,냉전,냉전 종식,냉전,냉전 종식,냉전
mrc-0-002191,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스
mrc-0-003951,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드
mrc-1-001272,프랑스,프랑스,프랑스,프랑스,프랑스
...,...,...,...,...,...
mrc-0-002989,재단 X,재단 X,재단 X,재단 X,재단 X
mrc-0-001804,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들
mrc-0-003411,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임
mrc-0-003436,디에프 항,주노 해변,디에프 항,주노 해변,디에프 항


In [21]:
column_name_list

['data0', 'data1', 'data2', 'data3', 'data4']

In [6]:
from collections import defaultdict

def count_list(data_list):
    # 개수를 key로 하여 data를 저장할 df
    count_df = defaultdict(list)
    
    # 중복 제거 list
    drop_duplicate_list = list(set(data_list))

    for element in drop_duplicate_list:
        data_cnt = data_list.count(element)
        # try:
        count_df[data_cnt].append(element)
        # except:
        #     count_df[data_cnt]=[element]
    
    return count_df

In [15]:
from datasets import load_from_disk, concatenate_datasets
from konlpy.tag import Mecab 
from tqdm.auto import tqdm
tokenizer = Mecab() 


def analysis_data():
    """
    train, validation answer의 형태소별 빈도수 저장
    """
    dataset = load_from_disk('../../data/train_dataset')
    train_dataset = dataset['train']
    val_dataset = dataset['validation']

    merged_dataset = concatenate_datasets([
        train_dataset.flatten_indices(),
        val_dataset.flatten_indices(),
    ])

    # print(merged_dataset.column_names)
    
    freq = defaultdict(int)
    for idx in tqdm(range(len(merged_dataset))):
        # print(merged_dataset['answers'][idx])

        results = tokenizer.pos(merged_dataset['answers'][idx]['text'][0])

        for result in results:
            freq[result[1]] += 1
    
    return freq
    
freq = analysis_data()

Loading cached processed dataset at ../../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


In [ ]:
def select_morpheme(answer_candidates):
    [tokenizer.pos(answer) for answer in answer_candidates]

In [25]:
"""
Hard Voting
"""

hard_voting = {}
# key = doc name
for key in key_list:
    # data 저장
    data_list = []
    for col in column_name_list:
        data_list.append(merge_df.loc[key][col])
    # data count
    count = count_list(data_list)

    answer_candidates = count[max(count.keys())]

    # 빈도수 동률 발생
    if len(answer_candidates) > 1:
        hard_voting[key] = 
    elif len(answer_candidates) == 1:
        hard_voting[key] = answer_candidates[0]
        

defaultdict(<class 'list'>, {2: ['40억년전 지구'], 1: ['사락사라', '지구', '별하늘계의 변경에 위치한 외계행성 사만(サマーン)']})
defaultdict(<class 'list'>, {3: ['냉전'], 2: ['냉전 종식']})
defaultdict(<class 'list'>, {5: ['대통령인 빌헬름 미클라스']})
defaultdict(<class 'list'>, {5: ['뉴질랜드']})
defaultdict(<class 'list'>, {5: ['프랑스']})
defaultdict(<class 'list'>, {2: ['아래턱'], 3: ['다리']})
defaultdict(<class 'list'>, {3: ['대의제도'], 1: ['91년 체제', '자유주의 원리']})
defaultdict(<class 'list'>, {5: ['하코네 산의 화산 폭발']})
defaultdict(<class 'list'>, {5: ['순조 11년(1811)']})
defaultdict(<class 'list'>, {5: ['고전도성 철']})
defaultdict(<class 'list'>, {4: ['리처드 말킨'], 1: ['시간 여행']})
defaultdict(<class 'list'>, {4: ['칼라치 전방 약250km 지점'], 1: ['칼라치']})
defaultdict(<class 'list'>, {5: ['롭 포드']})
defaultdict(<class 'list'>, {5: ['노스햄프턴 교회']})
defaultdict(<class 'list'>, {3: ['언니'], 2: ['한 언니']})
defaultdict(<class 'list'>, {5: ['석조 궁륭']})
defaultdict(<class 'list'>, {5: ['국민']})
defaultdict(<class 'list'>, {1: ['《자유》'], 4: ['‘자유’지에']})
defaultdict(<class 'list'>, {